In [14]:
import pandas as pd
import nltk
from nltk import ngrams
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import  CountVectorizer, TfidfVectorizer
#import pyLDAvis.gensim
import gensim
import pickle 
import pyLDAvis
pyLDAvis.enable_notebook()
import pyLDAvis.gensim
import os
import gensim.corpora as corpora
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/JaredJose/Roblox-Scraper/main/data/parentingComments.csv?token=GHSAT0AAAAAAB6VRXEZAMOYEJIRGAUK3HN4ZALOOTA')

In [16]:
df.head()

,url,author,date,timestamp,score,comment,comment_id,parent_id
0,https://www.reddit.com/r/Parenting/comments/yp...,EmotionalLaborQueen,2022-11-08,1.667928e+09,7,Minecraft. My kiddo is 9 now but started at 7 ...,ivkkezg,NaN
1,https://www.reddit.com/r/Parenting/comments/yp...,Solidsnakeerection,2022-11-08,1.667927e+09,3,From what I understand an Obby to be Little Bi...,ivkhhcx,NaN
2,https://www.reddit.com/r/Parenting/comments/yp...,Careless_Dependent94,2022-11-09,1.667952e+09,2,"Most scams aren’t in obbys, and if you dont wa...",ivm8nh6,NaN
3,https://www.reddit.com/r/Parenting/comments/yp...,tmarquez92,2022-11-08,1.667937e+09,1,My daughter's two favorites are PK XD and Play...,ivl67eh,NaN
4,https://www.reddit.com/r/Parenting/comments/yp...,leomercury,2022-11-09,1.667974e+09,1,maybe poptropica?,ivnh6qa,NaN


In [17]:
df['comment']

0       Minecraft. My kiddo is 9 now but started at 7 ...
1       From what I understand an Obby to be Little Bi...
2       Most scams aren’t in obbys, and if you dont wa...
3       My daughter's two favorites are PK XD and Play...
4                                       maybe poptropica?
                              ...                        
1524    Thanks! I’m always asking him “how would you f...
1525                                           Good luck!
1526    How about Roblox my kids are 5 and 6 and have ...
1527             I second Roblox. My 9 year old loves it.
1528    My 4 year old daughter understands death prett...
Name: comment, Length: 1529, dtype: object

In [18]:
df['comment'].str.count('game').sum()

622

In [19]:
text = " ".join(df['comment'].dropna().astype(str))
tokens = nltk.word_tokenize(text)
tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words and token.strip() != '']


bi_grams = ngrams(filtered_tokens, 2)

In [20]:
bi_grams_list = list(bi_grams)
bi_grams_list = [tuple(filter(None, bg)) for bg in bi_grams_list]
#print(bi_grams_list)

In [21]:
bi_grams_list1 = [' '.join(bi_gram) for bi_gram in bi_grams_list if '(' not in bi_gram and ')' not in bi_gram]

freq_dist = nltk.FreqDist(bi_grams_list1)
most_common = freq_dist.most_common(1)
print('The most common bi-gram is:', most_common[0][0], 'with a frequency of', most_common[0][1])

The most common bi-gram is: year old with a frequency of 133


In [22]:
vectorizer = CountVectorizer(ngram_range = (2, 2))
X1=vectorizer.fit_transform(bi_grams_list1)
features = (vectorizer.get_feature_names_out())

vectorizer = TfidfVectorizer(ngram_range = (2, 2))
tfidf = vectorizer.fit_transform(bi_grams_list1)


sums = tfidf.sum(axis = 0)
data1 = []
for col, term in enumerate(features):
    data1.append( (term, sums[0, col] ))
ranking = pd.DataFrame(data1, columns = ['term', 'rank'])
words = (ranking.sort_values('rank', ascending = False))
print ("\n\nWords : \n", words.head(7))



Words : 
               term   rank
44242     year old  133.0
5482         ca nt   82.0
33577  screen time   82.0
35824  sounds like   60.0
23057    make sure   39.0
41872  video games   37.0
43381        wo nt   35.0


In [23]:
df_ques = df['comment'].str.endswith('?')
df_true = df.loc[df_ques]
print(df_true['comment'])

4                                       maybe poptropica?
134     Isn't there a parent app that keeps things fro...
149     If Kylee has her own money, I say let them wor...
160                                 But did you buy a gf?
183     When our boys were some 3-5 yo we allowed them...
                              ...                        
1396    so there’s “downtime” which is basically no sc...
1400    Not unless the adults in the home are the ones...
1412    I feel like "grounding" is a bit too general a...
1488    Ease into it by using something like FaceTime ...
1509    Really? You can get scratch on pc. What kind o...
Name: comment, Length: 68, dtype: object


In [24]:
df['text_processed'] = df['comment'].apply(lambda x: re.sub('[,.!?]', '', x) if isinstance(x, str) else x)
df['text_processed'] = df['text_processed'].apply(lambda x: x.lower() if isinstance(x, str) else x)

df['text_processed'].head()

0    minecraft my kiddo is 9 now but started at 7 a...
1    from what i understand an obby to be little bi...
2    most scams aren’t in obbys and if you dont wan...
3    my daughter's two favorites are pk xd and play...
4                                     maybe poptropica
Name: text_processed, dtype: object

In [25]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = df.comment.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['minecraft', 'kiddo', 'started', 'loves']


In [26]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]
num_topics = 10
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
#s.makedirs('./results/ldavis_prepared_10')
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared
pyLDAvis.display(LDAvis_prepared)

/home/zhz094/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
